In [ ]:
from ecodonut.eco_frame import EcoFrameCalculator
from ecodonut.eco_frame import EcoFrame
import geopandas as gpd
import pickle
from ecodonut.utils.output import get_map
from ecodonut.eco_frame import mark_territory

In [ ]:
zhd = gpd.read_file('example_data/railway_rail.geojson')
dumps = gpd.read_file('example_data/landfill.geojson')
nature_reserve = gpd.read_file('example_data/nature_reserve.geojson')
industrial = gpd.read_file('example_data/industrial.geojson')
wood = gpd.read_file('example_data/wood.geojson')
water = gpd.read_file('example_data/water.geojson')

borders = gpd.read_file('example_data/region.geojson')

In [ ]:
layers = {
    'industrial': industrial,
    "gas_station": None,
    "landfill": dumps,
    'regional_roads': None,
    'federal_roads': None,
    "railway": zhd,
    "nature_reserve": nature_reserve,
    "water": water,
    'woods': wood
}

frame: EcoFrame = EcoFrameCalculator(territory=borders).evaluate_ecoframe(layers, min_layer_count=2, max_layer_count=8)


In [ ]:
pickle.dump(frame, open('tumen.pickle', 'wb'))
eco_frame: EcoFrame = pickle.load(open('tumen.pickle', 'rb'))

In [ ]:
get_map(eco_frame, -10, 10).save('map.html')

In [ ]:
zone = gpd.read_file('example_data/test_borders.geojson')

In [ ]:
mark_territory(eco_frame, zone.iloc[2:3])

In [ ]:
azs = gpd.read_file('example_data/fuel.geojson')
borders = gpd.read_file('example_data/region.geojson')
layers = {
    "gas_station": azs,
}
frame_add: EcoFrame = EcoFrameCalculator(territory=borders, settings_from=eco_frame).evaluate_ecoframe(layers)


In [ ]:
from ecodonut.eco_frame import concat_ecoframes
new_frame = concat_ecoframes(eco_frame,frame_add)

In [ ]:
get_map(new_frame, -10, 10).save('map_test.html')